### 最適化モデル作成の高度なテクニック

In [2]:
import numpy as np
from pulp import *
from ortoolpy import *

In [3]:
m=LpProblem() # 数理モデル
x1=addvars(10) # 一次元変数
x= addvar() # 0次元変数
z= addbinvar() # 0ー1変数
m += lpSum(x) # 目的関数

#### 変数をnp.arrayで作成する：Numpyのスライスなどの機能が使える

In [4]:
x2=addvars(2,3) # 2行3列の二次元の変数(リスト)
x3=np.array(x2) # 2行3列の二次元の変数(numpy)

In [5]:
x2

[[v000013, v000014, v000015], [v000016, v000017, v000018]]

In [6]:
x3

array([[v000013, v000014, v000015],
       [v000016, v000017, v000018]], dtype=object)

#### 結果をnp.vectorizeで取得する：内包表記より簡潔に記述でき、高速に動作する

In [9]:
# 内包表記
val = [value(v) for v in x1]
val

[None, None, None, None, None, None, None, None, None, None]

In [12]:
# np.vaectorize
val=np.vectorize(value)(x1)
val

array([None, None, None, None, None, None, None, None, None, None],
      dtype=object)

#### 元数の範囲を指定する：範囲が記述できる